In [1]:
import ray
import modules.network_extractor as net_extractor
import geopandas as geopd
from shapely import ops
from osmnx import settings
import pyproj
from pathlib import Path
import networkx as nx
import osmnx as ox
import shapely

ModuleNotFoundError: No module named 'modules'

In [3]:
# Change for your own data base path
data_base_path = "/home/user/Desktop/JP/street-network-indices/data"

# The extractor instance
extractor = net_extractor.NetworkExtractor()
extractor.DATA_BASE_PATH = data_base_path
 
# Custom OSMnx settings
settings.default_crs = "epsg:4326"

In [4]:
# the city name in lowercase and slug_case where the graphs are located
city_name = "sarajevo"

In [5]:
# Load graphs if they are already 
g_walk = extractor.load_graph(f'{city_name}/graph/walk_{city_name}')
g_bike = extractor.load_graph(f'{city_name}/graph/bike_{city_name}')
g_drive = extractor.load_graph(f'{city_name}/graph/drive_{city_name}')
g_public = extractor.load_graph(f'{city_name}/graph/public_{city_name}')

In [6]:
print(f"Pedestrian graph for {city_name}")
print(g_walk.number_of_edges())
print(g_walk.number_of_nodes())

Pedestrian graph for sarajevo
17154
13787


In [ ]:
walk_nodes, walk_edges = ox.graph_to_gdfs(g_walk)

In [89]:
target_geom = walk_edges.iloc[0].geometry

In [90]:
walk_edges

geometry  \
u           v           key                                                      
1893060772  251749740   0     LINESTRING (18.36522 43.81452, 18.3673 43.81386)   
            1893060770  0     LINESTRING (18.36766 43.81416, 18.3673 43.81386)   
            253044128   0    LINESTRING (18.36848 43.81316, 18.36773 43.813...   
251749740   1831919734  0    LINESTRING (18.36522 43.81452, 18.36508 43.814...   
            1831919255  0    LINESTRING (18.36522 43.81452, 18.3652 43.8144...   
...                                                                        ...   
12548766964 12548766968 0    LINESTRING (18.40596 43.86412, 18.40594 43.864...   
12547896602 12547896604 0    LINESTRING (18.4063 43.86404, 18.40627 43.8640...   
12547891698 12547924336 0    LINESTRING (18.40586 43.86323, 18.40589 43.863...   
            12547924307 0    LINESTRING (18.40649 43.86344, 18.40647 43.863...   
12547924336 12547891658 0    LINESTRING (18.40593 43.86319, 18.40592 43.863...   

                                 length                    name      highway  \
u           v           key                                                    
1893060772  251749740   0    182.286997  Kasindolskog bataljona     tertiary   
            1893060770  0     44.411913   Trg Miroslava Berjana  residential   
            253044128   0    122.456787  Kasindolskog bataljona     tertiary   
251749740   1831919734  0    146.032312  Kasindolskog bataljona     tertiary   
            1831919255  0     55.170440           Slavka Leovca  residential   
...                                 ...                     ...          ...   
12548766964 12548766968 0     24.542708                     NaN      footway   
12547896602 12547896604 0      9.840378                     NaN      footway   
12547891698 12547924336 0     15.580463                     NaN      footway   
            12547924307 0     51.578633                     NaN      footway   
12547924336 12547891658 0      8.300431                     NaN      footway   

                                bearing  foot footway  speed_kph  travel_time  \
u           v           key                                                     
1893060772  251749740   0    113.740102  None    None        5.0   131.246638   
            1893060770  0    220.612695  None    None        5.0    31.976577   
            253044128   0    309.365013  None    None        5.0    88.168887   
251749740   1831919734  0    294.298667  None    None        5.0   105.143265   
            1831919255  0    208.768237  None    None        5.0    39.722717   
...                                 ...   ...     ...        ...          ...   
12548766964 12548766968 0    265.151583  None    None        5.0    17.670750   
12547896602 12547896604 0    280.136176  None    None        5.0     7.085072   
12547891698 12547924336 0     15.186414  None    None        5.0    11.217934   
            12547924307 0    260.217062  None    None        5.0    37.136616   
12547924336 12547891658 0    307.894087  None    None        5.0     5.976310   

                                grade  grade_abs access sidewalk  
u           v           key                                       
1893060772  251749740   0   -0.027429   0.027429    NaN      NaN  
            1893060770  0   -0.022516   0.022516    NaN      NaN  
            253044128   0    0.016332   0.016332    NaN      NaN  
251749740   1831919734  0   -0.020543   0.020543    NaN      NaN  
            1831919255  0   -0.036251   0.036251    NaN      NaN  
...                               ...        ...    ...      ...  
12548766964 12548766968 0    0.203726   0.203726    NaN      NaN  
12547896602 12547896604 0    0.203244   0.203244    NaN      NaN  
12547891698 12547924336 0    0.000000   0.000000    NaN      NaN  
            12547924307 0    0.155103   0.155103    NaN      NaN  
12547924336 12547891658 0    0.000000   0.000000    NaN      NaN  

[17154 rows x 13 columns]

In [91]:
import time
buffered = target_geom.buffer(0.001)

In [112]:
# Save timestamp
start = time.time()


nearest_idx = walk_edges.reset_index(drop=True).sindex.query(buffered, sort=True, predicate="intersects")
#nearest_idx = walk_nodes.sindex.query(buffered, sort=True, predicate="intersects")
nearest = walk_edges.iloc[nearest_idx]
print(len(nearest))
# 1893060772	251749740


# Save timestamp
end = time.time()

print(end - start)

14
0.012668609619140625


In [113]:

list(nearest.index)

[(1893060772, 251749740, 0),
 (1893060772, 1893060770, 0),
 (1893060772, 253044128, 0),
 (251749740, 1831919734, 0),
 (251749740, 1831919255, 0),
 (251749740, 6363417717, 0),
 (1893060770, 253044113, 0),
 (1893060770, 6903919695, 0),
 (251749739, 1831919255, 0),
 (1831919255, 1831919254, 0),
 (12396747835, 12396747889, 0),
 (10311767571, 10311767570, 0),
 (12396747889, 12396747880, 0),
 (12396747889, 12396747886, 0)]

In [116]:


subg = g_walk.edge_subgraph(list(nearest.index))

for u,v in subg.edges():
    print(u,v)

6903919695 1893060770
253044113 1893060770
10311767570 10311767571
1831919254 1831919255
1831919255 251749740
1831919255 251749739
253044128 1893060772
1893060770 1893060772
1893060772 251749740
12396747880 12396747889
251749740 1831919734
251749740 6363417717
12396747886 12396747889
12396747889 12396747835


In [96]:
# Save timestamp
start = time.time()

matches = []
for n, data in g_walk.nodes(data=True):
    if shapely.intersects(buffered, shapely.Point(data["x"], data["y"])):
        matches.append(n)
print(len(matches))

# Save timestamp
end = time.time()

print(end - start)

9
0.10179948806762695


In [104]:
walk_nodes

,osmid,y,x,street_count,elevation,highway,crossing,railway,ref,geometry
osmid,,,,,,,,,,
1893060772,1893060772,43.813856,18.367296,3,542.0,NaN,NaN,NaN,NaN,POINT (18.3673 43.81386)
251749740,251749740,43.814515,18.365217,4,537.0,NaN,NaN,NaN,NaN,POINT (18.36522 43.81452)
1893060770,1893060770,43.814159,18.367657,3,541.0,NaN,NaN,NaN,NaN,POINT (18.36766 43.81416)
253044128,253044128,43.813157,18.368476,3,544.0,NaN,NaN,NaN,NaN,POINT (18.36848 43.81316)
253044119,253044119,43.813458,18.368832,4,545.0,NaN,NaN,NaN,NaN,POINT (18.36883 43.81346)
...,...,...,...,...,...,...,...,...,...,...
12547924307,12547924307,43.863443,18.406493,1,605.0,NaN,NaN,NaN,NaN,POINT (18.40649 43.86344)
12547891658,12547891658,43.863190,18.405935,1,597.0,NaN,NaN,NaN,NaN,POINT (18.40593 43.86319)
12547924335,12547924335,43.863269,18.405381,1,590.0,NaN,NaN,NaN,NaN,POINT (18.40538 43.86327)
